In [412]:
import pandas as pd
import numpy as np
from collections import defaultdict
from enum import Enum
import os
import re 
import json
from copy import deepcopy

#### Read all the json file from the igdb api

In [413]:
def load_files_to_df(keyword, directory):
    dfs = []
    print(os.getcwd()+directory)
    for filename in os.listdir(os.getcwd()+directory):
        if keyword in filename: 
            dfs.append(pd.read_json(f'{os.getcwd()+directory}\\{filename}'))
    print(f"{len(dfs)} files loaded with the keyword {keyword}")
    df = pd.concat(dfs)
    df.reset_index(inplace=True)
    return df

## Remove columns that wnot be used 

#### Columns that won't be used at all

In [414]:
games = load_files_to_df("file", "\data")
games = games.drop("created_at", axis=1).drop("external_games", axis=1).drop("release_dates", axis=1).drop("similar_games", axis=1).drop("updated_at", axis=1).drop("websites", axis=1).drop("parent_game", axis=1).drop("videos", axis=1).drop("game_engines", axis=1).drop("alternative_names", axis=1).drop("storyline", axis=1).drop("summary", axis=1).drop("checksum", axis=1).drop("artworks", axis=1).drop("cover", axis=1).drop("first_release_date", axis=1).drop("screenshots", axis=1).drop("slug", axis=1).drop("url", axis=1).drop("keywords", axis=1)
games.info()

C:\Users\ytagh\OneDrive\Documents\programming\python\WeGame\WeGameReccs\data
310 files loaded with the keyword file
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154871 entries, 0 to 154870
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   index                    154871 non-null  int64  
 1   id                       154871 non-null  int64  
 2   category                 154871 non-null  int64  
 3   game_modes               80017 non-null   object 
 4   genres                   115276 non-null  object 
 5   name                     154871 non-null  object 
 6   platforms                93408 non-null   object 
 7   status                   10199 non-null   float64
 8   tags                     120775 non-null  object 
 9   themes                   75298 non-null   object 
 10  aggregated_rating        13488 non-null   float64
 11  aggregated_rating_count  13488 non-null   float64
 12

In [415]:
games

,index,id,category,game_modes,genres,name,platforms,status,tags,themes,...,multiplayer_modes,franchise,expansions,dlcs,expanded_games,ports,remasters,remakes,standalone_expansions,forks
0,0,90269,0,[1],"[31, 32]",Nuclear 2050,[6],4.0,"[1, 268435487, 268435488]",[1],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40104,0,NaN,[5],Dogou Souken,[52],NaN,[268435461],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,24037,0,[1],"[9, 32]",Three Digits,[6],NaN,"[268435465, 268435488, 536871029]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,41825,1,[1],"[12, 31]",The Legend of Zelda: Breath of the Wild - The ...,[130],NaN,"[1, 17, 268435468, 268435487, 536871453]","[1, 17]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,68841,0,[2],"[15, 16]",Captivus,[6],4.0,"[18, 268435471, 268435472, 536871882, 536874083]",[18],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154866,495,100029,0,NaN,NaN,Frozen Dragon Gems Unlocked,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154867,496,96015,0,NaN,NaN,Blast Valley - Flip the Gun,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154868,497,98839,0,"[1, 2]","[9, 26, 33]",Party Trivia,"[48, 130]",NaN,"[40, 268435465, 268435482, 268435489]",[40],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154869,498,141039,1,"[1, 3]","[5, 31]",Sniper Elite 4: Deathstorm Part 3 - Obliteration,"[6, 48, 49]",NaN,"[1, 39, 268435461, 268435487]","[1, 39]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Remove non alphanumeric letters from names

In [416]:
def clear(string):
    """return: Returns the string, stripped of all letters except alphanum
    If the string doesnt contain alphanum or only conatins numbers, None is returned"""
    string = (re.sub("[^A-Za-z0-9]+", "", string)).lower()
    if (string.isdecimal()) or (len(string) == 0):
        return
    return string

In [417]:
games["name"] = games["name"].apply(clear)
games = games[games["name"].notna()]

In [418]:
games = games.reset_index(drop=True)

In [419]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154241 entries, 0 to 154240
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   index                    154241 non-null  int64  
 1   id                       154241 non-null  int64  
 2   category                 154241 non-null  int64  
 3   game_modes               79639 non-null   object 
 4   genres                   114754 non-null  object 
 5   name                     154241 non-null  object 
 6   platforms                93123 non-null   object 
 7   status                   10140 non-null   float64
 8   tags                     120244 non-null  object 
 9   themes                   75132 non-null   object 
 10  aggregated_rating        13484 non-null   float64
 11  aggregated_rating_count  13484 non-null   float64
 12  involved_companies       65676 non-null   object 
 13  rating                   27608 non-null   float64
 14  rati

In [420]:
games

,index,id,category,game_modes,genres,name,platforms,status,tags,themes,...,multiplayer_modes,franchise,expansions,dlcs,expanded_games,ports,remasters,remakes,standalone_expansions,forks
0,0,90269,0,[1],"[31, 32]",nuclear2050,[6],4.0,"[1, 268435487, 268435488]",[1],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40104,0,NaN,[5],dogousouken,[52],NaN,[268435461],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,24037,0,[1],"[9, 32]",threedigits,[6],NaN,"[268435465, 268435488, 536871029]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,41825,1,[1],"[12, 31]",thelegendofzeldabreathofthewildthemastertrials,[130],NaN,"[1, 17, 268435468, 268435487, 536871453]","[1, 17]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,68841,0,[2],"[15, 16]",captivus,[6],4.0,"[18, 268435471, 268435472, 536871882, 536874083]",[18],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154236,495,100029,0,NaN,NaN,frozendragongemsunlocked,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154237,496,96015,0,NaN,NaN,blastvalleyflipthegun,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154238,497,98839,0,"[1, 2]","[9, 26, 33]",partytrivia,"[48, 130]",NaN,"[40, 268435465, 268435482, 268435489]",[40],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154239,498,141039,1,"[1, 3]","[5, 31]",sniperelite4deathstormpart3obliteration,"[6, 48, 49]",NaN,"[1, 39, 268435461, 268435487]","[1, 39]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [421]:
t = games.sort_values(by='total_rating_count', ascending=False)

In [422]:
t

,index,id,category,game_modes,genres,name,platforms,status,tags,themes,...,multiplayer_modes,franchise,expansions,dlcs,expanded_games,ports,remasters,remakes,standalone_expansions,forks
99384,312,1942,0,[1],"[12, 31]",thewitcher3wildhunt,"[6, 48, 49, 130]",NaN,"[1, 17, 38, 268435468, 268435487, 536871041, 5...","[1, 17, 38]",...,NaN,NaN,"[12503, 13166]",[157580],NaN,NaN,NaN,NaN,NaN,NaN
143187,402,72,0,"[1, 2, 3, 4]","[5, 8, 9, 31]",portal2,"[3, 6, 9, 12, 14]",NaN,"[1, 18, 27, 268435461, 268435464, 268435465, 2...","[1, 18, 27]",...,"[11591, 11592, 11593, 11594, 11595]",NaN,NaN,"[99969, 106227, 114140]",NaN,NaN,NaN,NaN,NaN,NaN
143813,29,71,0,[1],"[5, 8, 9]",portal,"[3, 6, 9, 12, 14, 34]",NaN,"[1, 18, 27, 268435461, 268435464, 268435465, 5...","[1, 18, 27]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117360,363,732,0,"[1, 3]","[5, 10, 31]",grandtheftautosanandreas,"[6, 8, 9, 11, 12, 14, 34, 39, 48]",NaN,"[1, 32, 38, 268435461, 268435466, 268435487, 5...","[1, 32, 38]",...,[1858],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113364,352,1009,0,"[1, 2]","[5, 31]",thelastofus,[9],NaN,"[1, 19, 21, 23, 268435461, 268435487, 53687097...","[1, 19, 21, 23]",...,NaN,NaN,[11790],NaN,NaN,NaN,[6036],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154236,495,100029,0,NaN,NaN,frozendragongemsunlocked,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154237,496,96015,0,NaN,NaN,blastvalleyflipthegun,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154238,497,98839,0,"[1, 2]","[9, 26, 33]",partytrivia,"[48, 130]",NaN,"[40, 268435465, 268435482, 268435489]",[40],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154239,498,141039,1,"[1, 3]","[5, 31]",sniperelite4deathstormpart3obliteration,"[6, 48, 49]",NaN,"[1, 39, 268435461, 268435487]","[1, 39]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [423]:
t.to_json(f"{os.getcwd()}\dataClean\games02.json")

## Replace numerical values with comma seperated strings

In [388]:
def get_nums_from_strArr(strArr):
    if strArr is not None:
        nums = strArr.replace("[", "").replace("]", "").strip()
        if len(nums) == 1:
            return int(nums)
        else:
            nums = split(", ")
            nums = [int(nums) for num in  nums]
            return nums
    else:
        return None

In [389]:
def replace_with_str_at_i(num_to_str_map, in_arr):
    out_arr = [] 
    if type(in_arr)==float:
        return np.nan
    for entry in in_arr:
        out_arr.append(num_to_str_map[str(entry)])
    #print(f"{in_arr} has been turned to {out_arr}")
    return out_arr

The only columns that will be used for the model are **game_modes**, **genres**, **theme**, and **player perspective**. name will also be used, but has to anly have alphabet letters and numbers.

#### Game Modes

In [390]:
GameModes = {"1":"Single player", "2":"Multiplayer", "3":"Co-operative", "4":"Split Screen", "5":"Massively Multiplayer Online (MMO)", "6":"Battle Royale"}
#games["game_modes"].apply(replace_with_str_at_i, GameModes)
l = list(deepcopy(games["game_modes"]))
l = [replace_with_str_at_i(GameModes, item) for item in l]
print(pd.Series(l))
games.loc[:, "game_modes"] = pd.Series(l)

0                       [Single player]
1                                   NaN
2                       [Single player]
3                       [Single player]
4                         [Multiplayer]
                      ...              
154236                              NaN
154237                              NaN
154238     [Single player, Multiplayer]
154239    [Single player, Co-operative]
154240                              NaN
Length: 154241, dtype: object


In [391]:
games.head(10)

,index,id,category,game_modes,genres,name,platforms,status,tags,themes,...,multiplayer_modes,franchise,expansions,dlcs,expanded_games,ports,remasters,remakes,standalone_expansions,forks
0,0,90269,0,[Single player],"[31, 32]",nuclear2050,[6],4.0,"[1, 268435487, 268435488]",[1],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40104,0,NaN,[5],dogousouken,[52],NaN,[268435461],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,24037,0,[Single player],"[9, 32]",threedigits,[6],NaN,"[268435465, 268435488, 536871029]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,41825,1,[Single player],"[12, 31]",thelegendofzeldabreathofthewildthemastertrials,[130],NaN,"[1, 17, 268435468, 268435487, 536871453]","[1, 17]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,68841,0,[Multiplayer],"[15, 16]",captivus,[6],4.0,"[18, 268435471, 268435472, 536871882, 536874083]",[18],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,33284,0,[Single player],[32],onewaytoexit,[6],NaN,"[1, 268435488]",[1],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,104748,0,NaN,NaN,spacestationbuildyourowniss,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,146815,0,[Single player],[9],parkinglotmaze,[6],NaN,[268435465],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,85450,0,NaN,NaN,transformersprimethegame,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,89616,0,NaN,NaN,bubblewhirlshooter,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Genres

In [392]:
df = pd.read_json("data\genres.json")

ids = list(df["id"])
genres = list(df["name"])

genres_dict = []
for id_, genre in zip(ids, genres):
    genres_dict.append((str(id_), genre))
genres_dict = dict(genres_dict)

print(genres_dict)

{'4': 'Fighting', '5': 'Shooter', '7': 'Music', '8': 'Platform', '9': 'Puzzle', '10': 'Racing', '11': 'Real Time Strategy (RTS)', '12': 'Role-playing (RPG)', '13': 'Simulator', '14': 'Sport', '15': 'Strategy', '16': 'Turn-based strategy (TBS)', '24': 'Tactical', '26': 'Quiz/Trivia', '25': "Hack and slash/Beat 'em up", '30': 'Pinball', '31': 'Adventure', '33': 'Arcade', '34': 'Visual Novel', '32': 'Indie', '35': 'Card & Board Game', '36': 'MOBA', '2': 'Point-and-click'}


In [393]:
l = list(deepcopy(games["genres"]))
l = [replace_with_str_at_i(genres_dict, item) for item in l]
games.loc[:, "genres"] = pd.Series(l)

In [394]:
games

,index,id,category,game_modes,genres,name,platforms,status,tags,themes,...,multiplayer_modes,franchise,expansions,dlcs,expanded_games,ports,remasters,remakes,standalone_expansions,forks
0,0,90269,0,[Single player],"[Adventure, Indie]",nuclear2050,[6],4.0,"[1, 268435487, 268435488]",[1],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40104,0,NaN,[Shooter],dogousouken,[52],NaN,[268435461],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,24037,0,[Single player],"[Puzzle, Indie]",threedigits,[6],NaN,"[268435465, 268435488, 536871029]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,41825,1,[Single player],"[Role-playing (RPG), Adventure]",thelegendofzeldabreathofthewildthemastertrials,[130],NaN,"[1, 17, 268435468, 268435487, 536871453]","[1, 17]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,68841,0,[Multiplayer],"[Strategy, Turn-based strategy (TBS)]",captivus,[6],4.0,"[18, 268435471, 268435472, 536871882, 536874083]",[18],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154236,495,100029,0,NaN,NaN,frozendragongemsunlocked,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154237,496,96015,0,NaN,NaN,blastvalleyflipthegun,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154238,497,98839,0,"[Single player, Multiplayer]","[Puzzle, Quiz/Trivia, Arcade]",partytrivia,"[48, 130]",NaN,"[40, 268435465, 268435482, 268435489]",[40],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154239,498,141039,1,"[Single player, Co-operative]","[Shooter, Adventure]",sniperelite4deathstormpart3obliteration,"[6, 48, 49]",NaN,"[1, 39, 268435461, 268435487]","[1, 39]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Themes

In [395]:
print(f"{os.getcwd()}\data\\themes.json")
df = pd.read_json(f"{os.getcwd()}\data\\themes.json")

ids = list(df["id"])
themes = list(df["name"])

themes_dict = []
for id_, theme in zip(ids, themes):
    themes_dict.append((str(id_), theme))
themes_dict = dict(themes_dict)

print(themes_dict)

C:\Users\ytagh\OneDrive\Documents\programming\python\WeGame\WeGameReccs\data\themes.json
{'20': 'Thriller', '18': 'Science fiction', '1': 'Action', '19': 'Horror', '21': 'Survival', '17': 'Fantasy', '22': 'Historical', '23': 'Stealth', '27': 'Comedy', '28': 'Business', '31': 'Drama', '32': 'Non-fiction', '35': 'Kids', '33': 'Sandbox', '38': 'Open world', '39': 'Warfare', '41': '4X (explore, expand, exploit, and exterminate)', '34': 'Educational', '43': 'Mystery', '40': 'Party', '44': 'Romance', '42': 'Erotic'}


In [396]:
l = list(deepcopy(games["themes"]))
l = [replace_with_str_at_i(themes_dict, item) for item in l]
games.loc[:, "themes"] = pd.Series(l)

In [397]:
games

,index,id,category,game_modes,genres,name,platforms,status,tags,themes,...,multiplayer_modes,franchise,expansions,dlcs,expanded_games,ports,remasters,remakes,standalone_expansions,forks
0,0,90269,0,[Single player],"[Adventure, Indie]",nuclear2050,[6],4.0,"[1, 268435487, 268435488]",[Action],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40104,0,NaN,[Shooter],dogousouken,[52],NaN,[268435461],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,24037,0,[Single player],"[Puzzle, Indie]",threedigits,[6],NaN,"[268435465, 268435488, 536871029]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,41825,1,[Single player],"[Role-playing (RPG), Adventure]",thelegendofzeldabreathofthewildthemastertrials,[130],NaN,"[1, 17, 268435468, 268435487, 536871453]","[Action, Fantasy]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,68841,0,[Multiplayer],"[Strategy, Turn-based strategy (TBS)]",captivus,[6],4.0,"[18, 268435471, 268435472, 536871882, 536874083]",[Science fiction],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154236,495,100029,0,NaN,NaN,frozendragongemsunlocked,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154237,496,96015,0,NaN,NaN,blastvalleyflipthegun,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154238,497,98839,0,"[Single player, Multiplayer]","[Puzzle, Quiz/Trivia, Arcade]",partytrivia,"[48, 130]",NaN,"[40, 268435465, 268435482, 268435489]",[Party],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154239,498,141039,1,"[Single player, Co-operative]","[Shooter, Adventure]",sniperelite4deathstormpart3obliteration,"[6, 48, 49]",NaN,"[1, 39, 268435461, 268435487]","[Action, Warfare]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### player_perspectives

In [398]:
print(f"{os.getcwd()}\data\player_perspectives.json")
df = pd.read_json(f"{os.getcwd()}\data\player_perspectives.json")

ids = list(df["id"])
povs = list(df["name"])

pov_dict = []
for id_, pov in zip(ids, povs):
    pov_dict.append((str(id_), pov))
pov_dict = dict(pov_dict)

print(pov_dict)

C:\Users\ytagh\OneDrive\Documents\programming\python\WeGame\WeGameReccs\data\player_perspectives.json
{'1': 'First person', '2': 'Third person', '3': 'Bird view / Isometric', '5': 'Text', '4': 'Side view', '7': 'Virtual Reality', '6': 'Auditory'}


In [399]:
l = list(deepcopy(games["player_perspectives"]))
l = [replace_with_str_at_i(pov_dict, item) for item in l]
games.loc[:, "player_perspectives"] = pd.Series(l)

In [400]:
games.head(20)

,index,id,category,game_modes,genres,name,platforms,status,tags,themes,...,multiplayer_modes,franchise,expansions,dlcs,expanded_games,ports,remasters,remakes,standalone_expansions,forks
0,0,90269,0,[Single player],"[Adventure, Indie]",nuclear2050,[6],4.0,"[1, 268435487, 268435488]",[Action],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40104,0,NaN,[Shooter],dogousouken,[52],NaN,[268435461],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,24037,0,[Single player],"[Puzzle, Indie]",threedigits,[6],NaN,"[268435465, 268435488, 536871029]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,41825,1,[Single player],"[Role-playing (RPG), Adventure]",thelegendofzeldabreathofthewildthemastertrials,[130],NaN,"[1, 17, 268435468, 268435487, 536871453]","[Action, Fantasy]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,68841,0,[Multiplayer],"[Strategy, Turn-based strategy (TBS)]",captivus,[6],4.0,"[18, 268435471, 268435472, 536871882, 536874083]",[Science fiction],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,33284,0,[Single player],[Indie],onewaytoexit,[6],NaN,"[1, 268435488]",[Action],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,104748,0,NaN,NaN,spacestationbuildyourowniss,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,146815,0,[Single player],[Puzzle],parkinglotmaze,[6],NaN,[268435465],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,85450,0,NaN,NaN,transformersprimethegame,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,89616,0,NaN,NaN,bubblewhirlshooter,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
games.to_json(f"{os.getcwd()}\dataClean\games01.json")